In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from scripts.load_data import *
from scripts.preprocess import *
from scripts.helper import *
from scripts.applicant_prediction import *
from scripts.transform_data import *
from scripts.predict_sarima import *

In [3]:
data_individual, data_cumulative, data_student_numbers, data_latest, data_distances, data_weighted_ensemble = load_data()

In [4]:
print(data_individual.columns)
print(data_cumulative.columns)

Index(['Sleutel', 'Datum Verzoek Inschr', 'Ingangsdatum', 'Collegejaar',
       'Datum intrekking vooraanmelding', 'Inschrijfstatus', 'Faculteit',
       'Examentype', 'Croho', 'Croho groepeernaam', 'Opleiding',
       'Hoofdopleiding', 'Eerstejaars croho jaar',
       'Is eerstejaars croho opleiding', 'Is hogerejaars', 'BBC ontvangen',
       'Type vooropleiding', 'Nationaliteit', 'EER', 'Geslacht',
       'Geverifieerd adres postcode', 'Geverifieerd adres plaats',
       'Geverifieerd adres land', 'Studieadres postcode', 'Studieadres land',
       'School code eerste vooropleiding', 'School eerste vooropleiding',
       'Plaats code eerste vooropleiding', 'Land code eerste vooropleiding',
       'Aantal studenten'],
      dtype='object')
Index(['Korte naam instelling', 'Collegejaar', 'Weeknummer rapportage',
       'Weeknummer', 'Faculteit', 'Type hoger onderwijs', 'Groepeernaam Croho',
       'Naam Croho opleiding Nederlands', 'Croho', 'Herkomst',
       'Gewogen vooraanmelders', 'O

In [5]:
data_individual = read_and_preprocess(data_individual, data_distances, FilterStudentYear.VOLUME)
print(data_individual.columns)

Index(['Datum Verzoek Inschr', 'Collegejaar',
       'Datum intrekking vooraanmelding', 'Inschrijfstatus', 'Faculteit',
       'Examentype', 'Croho groepeernaam', 'Opleiding', 'Hoofdopleiding',
       'Is eerstejaars croho opleiding', 'Is hogerejaars',
       'Type vooropleiding', 'Nationaliteit', 'EER', 'Geslacht',
       'Geverifieerd adres postcode', 'Geverifieerd adres plaats',
       'Geverifieerd adres land', 'Studieadres postcode', 'Studieadres land',
       'School code eerste vooropleiding', 'School eerste vooropleiding',
       'Plaats code eerste vooropleiding', 'Land code eerste vooropleiding',
       'Sleutel_count', 'Weeknummer', 'Herkomst', 'is_numerus_fixus',
       'Afstand', 'Deadlineweek'],
      dtype='object')


In [6]:
predict_year = 2023
predict_week = 14

max_year = data_individual["Collegejaar"].max()
max_week = get_max_week(data_individual[data_individual["Collegejaar"] == predict_year]["Weeknummer"])

In [7]:
predicties = predict_preapplication(data_individual, predict_year, max_year, predict_week)

In [8]:
print(data_individual)
print(data_individual.columns)

       Datum Verzoek Inschr  Collegejaar  Datum intrekking vooraanmelding  \
0                  06-08-13         2013                              NaN   
1                  03-09-13         2013                              NaN   
4                  18-06-22         2022                              NaN   
5                  16-06-23         2023                              NaN   
6                  07-07-13         2013                              NaN   
...                     ...          ...                              ...   
475447             09-11-16         2017                             36.0   
475451             01-10-16         2017                             14.0   
475452             01-04-15         2015                             26.0   
475458             29-04-23         2023                              NaN   
475459             15-01-24         2024                              NaN   

                 Inschrijfstatus Faculteit Examentype  \
0                 

In [9]:
data_individual.loc[(data_individual["Collegejaar"] == predict_year) & (data_individual["Weeknummer"].isin(get_weeks_list(predict_week))), 'Inschrijvingen_predictie'] = predicties
print(data_individual)
print(data_individual.columns)

       Datum Verzoek Inschr  Collegejaar  Datum intrekking vooraanmelding  \
0                  06-08-13         2013                              NaN   
1                  03-09-13         2013                              NaN   
4                  18-06-22         2022                              NaN   
5                  16-06-23         2023                              NaN   
6                  07-07-13         2013                              NaN   
...                     ...          ...                              ...   
475447             09-11-16         2017                             36.0   
475451             01-10-16         2017                             14.0   
475452             01-04-15         2015                             26.0   
475458             29-04-23         2023                              NaN   
475459             15-01-24         2024                              NaN   

                 Inschrijfstatus Faculteit Examentype  \
0                 

In [10]:
data_individual = transform(data_individual, predict_year, max_week)
print(data_individual)
print(data_individual.columns)

       Collegejaar Faculteit  Herkomst Examentype  \
0             2013      FFTR       EER   Bachelor   
1             2013      FFTR       EER   Bachelor   
2             2013      FFTR       EER   Bachelor   
3             2013      FFTR       EER     Master   
4             2013      FFTR       EER     Master   
...            ...       ...       ...        ...   
13619         2023       RDA        NL     Master   
13620         2023       RDA        NL     Master   
13621         2023       RDA  Niet-EER     Master   
13622         2023       RDA  Niet-EER     Master   
13623         2023       RDA  Niet-EER     Master   

                                    Croho groepeernaam Weeknummer  \
0                                          B Filosofie         39   
1                               B Religiewetenschappen         39   
2                                          B Theologie         39   
3                                    M Filosofie (1jr)         39   
4                 

In [11]:
data_individual = transform_data(data_individual, targ_col='Cumulative_sum_within_year')
print(data_individual)
print(data_individual.columns)

      Collegejaar Faculteit  Herkomst Examentype  \
0            2013      FFTR       EER   Bachelor   
1            2013      FFTR       EER   Bachelor   
2            2013      FFTR       EER   Bachelor   
3            2013      FFTR       EER     Master   
4            2013      FFTR       EER     Master   
...           ...       ...       ...        ...   
2870         2023       RDA        NL     Master   
2871         2023       RDA        NL     Master   
2872         2023       RDA  Niet-EER     Master   
2873         2023       RDA  Niet-EER     Master   
2874         2023       RDA  Niet-EER     Master   

                                   Croho groepeernaam        39        40  \
0                                         B Filosofie  0.000000  0.000000   
1                              B Religiewetenschappen  0.000000  0.000000   
2                                         B Theologie  0.000000  0.000000   
3                                   M Filosofie (1jr)  0.000000  0.

In [12]:
dict = {'Collegejaar': [], 'Weeknummer': [], 'Faculteit': [], 'Examentype': [], 'Herkomst': [], 'Croho groepeernaam': [], 'SARIMA_individual': []}

for index, row in data_individual[data_individual["Collegejaar"] == predict_year].iterrows():
    sarima_ind = predict_with_sarima(data_individual, None, row["Croho groepeernaam"], row["Herkomst"], predict_week, predict_year, max_year)

    dict['Collegejaar'].append(predict_year)
    dict['Weeknummer'].append(predict_week)
    dict['Faculteit'].append(row["Faculteit"])
    dict['Examentype'].append(row["Examentype"])
    dict['Herkomst'].append(row["Herkomst"])
    dict['Croho groepeernaam'].append(row["Croho groepeernaam"])
    dict['SARIMA_individual'].append(sarima_ind)

dict = pd.DataFrame(dict)
dict.to_excel("individual_output.xlsx", index=False)